In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [37]:
pd.set_option('display.max_columns', None)

URL = 'https://www.fantasypros.com/nfl/reports/snap-count-analysis/{pos}.php'

pos_arr = ['qb', 'wr', 'rb', 'te']

season_df = pd.DataFrame()

for pos in pos_arr:
    res = requests.get(URL.format(pos=pos))
    soup = bs(res.content, 'html.parser')
    table = soup.find('table', {'id': 'data'})
    df = pd.read_html(str(table))[0]

    df['FPPerSnap'] = df['Fantasy Pts']/df['Snaps']
    df['TeamSnaps'] = (df['Snaps']/(df['Snap %']*0.01))
    df['Pos'] = pos.upper()
    
    df = df.rename(columns={
        'Player':'Name',
        'Games':'G',
        'Snap %': 'Snap%',
        'Snaps/Gm':'Per Game',
        'Rush %':'RushPerc',
        'Tgt %':'TargetP',
        'Touch %': 'TouchPerc',
        'Fantasy Pts':'FP'})
    df.drop(columns=['Util %', 'Pts/100 Snaps'], inplace=True)
    df = df.reindex(columns=[
        'Pos', 'Team', 'Name', 'G', 'Snaps', 'TeamSnaps', 'Snap%', 'RushPerc',\
            'TargetP', 'TouchPerc', 'FP', 'FPPerSnap' 
            ])
    
    season_df = pd.concat([season_df, df])

In [39]:
season_df.to_csv('/Users/brandynklee/Python_Projects/fantasy_footballl_22/data/snapcount_fp_2021.csv', index=False)